In [ ]:
import numpy as np
import serial
import serial.tools.list_ports
import platform
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import struct
import time

In [ ]:
def list_ports():
    devOS = platform.system()
    print("Detected", devOS, "Computer")
    ports = []
    if devOS == "Windows":
        ports = serial.tools.list_ports.comports()
    print("Listing all Serial Ports:")
    for p in ports:
        print(p)
        
list_ports()

In [ ]:
port = input("serial port 1:")
s1 = serial.Serial(port)
s1.flush()
port = input("serial port 2:")
s2 = serial.Serial(port)
s2.flush()

In [ ]:
def set_chirp_length(s=s1, t=1e-3):
    ba = bytearray(struct.pack("f", t))
    msg = b'0'+bytes(ba)
    s.write(msg)
    done = False
    while not done:
        msg = s.readline()
        print(msg)
        if msg.startswith(b'ack'):
            done = True

def set_start(s=s1, f=20000):
    msg = b'1'+(f).to_bytes(2, byteorder='little')
    s.write(msg)
    done = False
    while not done:
        msg = s.readline()
        print(msg)
        if msg.startswith(b'ack'):
            done = True

def set_stop(s=s1, f=20000):
    msg = b'2'+(f).to_bytes(2, byteorder='little')
    s.write(msg)
    done = False
    while not done:
        msg = s.readline()
        print(msg)
        if msg.startswith(b'ack'):
            done = True
                       
def get_size(s, string):
    done = False
    while not done:
        msg = s.readline()
        # print(msg)
        if msg.startswith(string.encode()):
            done =  True
    size = msg.decode().split(':')[-1]
    return int(size)

def get_mic(s=s1):
    size = get_size(s, 'left')
    l = s.read((size<<1))
    size = get_size(s, 'right')
    r = s.read((size<<1))
    msg = s.readline()
    if len(msg)!=2:
        print("problem")
    l = np.frombuffer(l, dtype=np.int16)
    r = np.frombuffer(r, dtype=np.int16)
    
    return l,r

def flush_serial(s=s1):
    while True:
        print(s.readline())

In [ ]:
freq_start =  20000
freq_stop = 20000
pulse_width = 1e-3

set_chirp_length(s1, pulse_width)
set_start(s=s1, f=freq_start)
set_stop(s=s1, f=freq_stop)

In [ ]:
s1.write(b'a')
s2.write(b'l')
time.sleep(0.1)
l,r = get_mic(s1)
s2.write(b't')
l2,r2 = get_mic(s2)
fig = go.Figure()
fs = 19e6/32/6;
dt = 1/fs;
x = np.arange(len(l))
trace_left = go.Scatter(x=x, y=l, mode='lines+markers')
fig.add_trace(trace_left)
trace_right = go.Scatter(x=x, y=r, mode='lines+markers')
fig.add_trace(trace_right)
trace_left = go.Scatter(x=x, y=l2, mode='lines+markers')
fig.add_trace(trace_left)
trace_right = go.Scatter(x=x, y=r2, mode='lines+markers')
fig.add_trace(trace_right)
fig.show()

In [ ]:
timerFreq = 19e6
top_start = int(timerFreq / freq_start);  # period of start frequency
top_stop = int(timerFreq / freq_stop);  # period of stop frequency
N_waves = int(pulse_width * (freq_stop+freq_start)/2);
top_step = (freq_start - freq_stop) / (freq_start * freq_stop) * timerFreq / (N_waves - 1);

top_start, top_stop, top_step, N_waves

list_periods = []
for i in range(N_waves):
    period = top_start + int(i*top_step)
    list_periods.append(period)
list_periods =  np.array(list_periods)

list_periods

chirp = []
for period in list_periods:
    chirp = np.hstack([chirp, np.ones(int(period/2)), np.zeros(period-int(period/2))])

offset = 0
chirp_pdm = chirp[offset::(32*6)]

In [ ]:

fig = go.Figure()
fs = 19e6/32/6;
dt = 1/fs;
x = np.arange(len(l))
trace_left = go.Scatter(x=x, y=np.correlate(l, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_left)
trace_right = go.Scatter(x=x, y=np.correlate(r, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_right)
trace_left = go.Scatter(x=x, y=np.correlate(l2, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_left)
trace_right = go.Scatter(x=x, y=np.correlate(r2, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_right)
fig

In [ ]:
#reverse chirp
freq_start =  30000
freq_stop = 30000
pulse_width = 2.5e-3

timerFreq = 19e6
top_start = int(timerFreq / freq_start);  # period of start frequency
top_stop = int(timerFreq / freq_stop);  # period of stop frequency
N_waves = int(pulse_width * (freq_stop+freq_start)/2);
top_step = (freq_start - freq_stop) / (freq_start * freq_stop) * timerFreq / (N_waves - 1);

top_start, top_stop, top_step, N_waves

list_periods = []
for i in range(N_waves):
    period = top_start + int(i*top_step)
    list_periods.append(period)
list_periods =  np.array(list_periods)

list_periods

chirp = []
for period in list_periods:
    chirp = np.hstack([chirp, np.ones(int(period/2)), np.zeros(period-int(period/2))])

offset = 0
chirp_pdm = chirp[offset::(32*6)]

In [ ]:

fig = go.Figure()
fs = 19e6/32/6;
dt = 1/fs;
x = np.arange(len(l))
trace_left = go.Scatter(x=x, y=np.correlate(l, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_left)
trace_right = go.Scatter(x=x, y=np.correlate(r, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_right)
trace_left = go.Scatter(x=x, y=np.correlate(l2, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_left)
trace_right = go.Scatter(x=x, y=np.correlate(r2, chirp_pdm, 'valid'), mode='lines+markers')
fig.add_trace(trace_right)
fig

In [ ]:
# this cell doesn't seem to want to run well

#  play chirp, get periods
# set_start(f=20000)
# set_stop(f=22000)
# pw=1e-3
# set_chirp_length(pw)
s.write(b'c')
print(s.readline())
s.write(b'p')
size = get_size('top_values:')
raw = s.read(size)
print(s.readline())
pulses = np.frombuffer(raw, dtype=np.int32)
pulses[pulses>0].sum() - 19e6*pw

In [ ]:
pulses[pulses>0]

In [ ]:
set_start(f=20000)
set_stop(f=30000)
pw=5e-3
set_chirp_length(pw)
s.write(b'c')

In [ ]:
while True:
    print(s2.readline())

In [ ]:
s1.write(b'l')
s1.write(b't')

In [ ]:
L = np.fft.rfft(l)
L2 = np.fft.rfft(l2)

In [ ]:
plt.semilogy(np.abs(L))
plt.semilogy(np.abs(L2))

In [ ]:
a = np.arange(100)
a.shape=(-1,10)
a.max(axis=1)

In [ ]:
b = a.cumsum()

In [ ]:
b[10:] - b[:-10]

In [ ]:
a = np.arange(4)
b = a+1

In [ ]:
for (aa,bb) in zip(a,b):
    print(aa,bb)